In [1]:
import json
import pandas as pd
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

# **CREAR ÍNDICE**

In [2]:
index_mapping = {
    "mappings": {
        "properties": {
            "ranking_date": {"type": "date", "format": "yyyy-MM-dd"},
            "pilot_name": {"type": "keyword"},
            "position": {"type": "integer"},
            "pilot_review": {"type": "text"},
            "ranking_link": {"type": "text"},
        }
    }
}

try:
    es.indices.create(index="my_index", body=index_mapping)
except Exception as e:
    print(f"Error: {e}")

Error: BadRequestError(400, 'resource_already_exists_exception', 'index [my_index/91JVpV3lQuy1-gJCLeUzUg] already exists')


In [3]:
# es.indices.delete(index='my_index')

# **INDEXAR CSV**

In [ ]:
csv = pd.read_csv("data/unstructured/rankings_info.csv")
df = pd.DataFrame(csv)

In [5]:
for index in range(len(df)):
    doc = df.iloc[index].to_dict()
    res_insert = es.index(index="my_index", id=index+1, document=doc)

In [6]:
print(json.dumps(res_insert.body, indent=3)) # el último insertado

{
   "_index": "my_index",
   "_type": "_doc",
   "_id": "940",
   "_version": 2,
   "result": "updated",
   "_shards": {
      "total": 2,
      "successful": 1,
      "failed": 0
   },
   "_seq_no": 2819,
   "_primary_term": 1
}


In [7]:
es.get(index='my_index', id=455)

ObjectApiResponse({'_index': 'my_index', '_type': '_doc', '_id': '455', '_version': 3, '_seq_no': 2334, '_primary_term': 1, 'found': True, '_source': {'ranking_date': '2022-11-23', 'pilot_name': 'Esteban Ocon', 'position': 5, 'pilot_review': 'Esteban Ocon qualified a solid eighth behind midfield rival Norris and the Alpine driver pretty much stayed right there for most of the race, his pace sufficient to prevent Vettel and team mate Fernando Alonso from catching up – but insufficient to catch Norris ahead. What was destined to be a P8 finish turned into P7 when Hamilton retired. ', 'ranking_link': 'https://www.formula1.com/en/latest/article/power-rankings-the-scores-come-in-from-the-final-race-of-2022-in-abu-dhabi.6cTDupNjqw80BGLYTEAymr'}})

# **CONSULTAS**

In [14]:
res = es.search(index='my_index', body={
     'query': {
         'range': {
             'position': {
                 'gte': 1,
                 'lte': 3
             }
         }
     },
     'size': 1
})

# print(json.dumps(res.body["hits"]["hits"][0]["_source"], indent=4))
print(json.dumps(res.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 283,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "my_index",
                "_type": "_doc",
                "_id": "1",
                "_score": 1.0,
                "_source": {
                    "ranking_date": "2024-10-23",
                    "pilot_name": "Charles Leclerc",
                    "position": 1,
                    "pilot_review": "Charles Leclerc appeared to be slightly on the back foot in the Ferrari intra-team battle after Saturday\u2019s action in Austin, having lost out to Carlos Sainz during the Sprint race and qualifying for the main event. However, the Monegasque came back fighting to brilliantly go from fourth to first at Turn 1 and then storm clear of the field. In Jenson 